# Stochastic Variational GP

**Warning**: Need to use GPUs in order for this to work.

In [39]:
import sys

# Add the path to the models
sys.path.insert(0, '/Users/eman/Documents/code_projects/ml4ocean')
sys.path.insert(0, '/home/emmanuel/projects/2019_ocean')
from src.models.utils import MultiTaskGP
from src.models.gpflow_gpu import SVGP, MOSVGP

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import time as time
import numpy as np
import gpflow
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [45]:
# Make Fake Dataset
X, y = make_regression(
    n_samples=10000, 
    n_features=10,    # Total Features
    n_informative=3,   # Informative Features 
    n_targets=10,
    bias=10,
    noise=0.8,
    random_state=123

)
train_size = 3000

# Training and Testing
xtrain, xtest, ytrain, ytest = train_test_split(
    X, y, train_size=train_size, random_state=123
)

xtrain.shape, ytrain.shape

((3000, 10), (3000, 10))

In [31]:
import tensorflow as tf

In [32]:
gpflow.__version__

'1.3.0'

In [33]:
tf.__version__

'1.13.1'

In [34]:
tf.train

<module 'tensorflow._api.v1.train' from '/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/tensorflow/_api/v1/train/__init__.py'>

### SVGP Algorithm

In [56]:
# Test Arguments to Ensure it works
class TestArgs:
    num_inducing = 100
    iterations = 10
    small_iterations = 10
    adam_lr = 0.01
    gamma = 0.1
    minibatch_size = 100
    initial_likelihood_var = 0.01
    seed = 0

# Better training arguments
class Args:
    num_inducing = 100
    iterations = 10000
    small_iterations = 1000
    adam_lr = 0.01
    gamma = 0.1
    minibatch_size = 1000
    initial_likelihood_var = 0.01

In [57]:
# Good practice
gpflow.reset_default_graph_and_session()

In [58]:
# Initialize Model
gp_model = SVGP(
    num_inducing=TestArgs.num_inducing,
    iterations=TestArgs.iterations,
    small_iterations=TestArgs.small_iterations,
    adam_lr=TestArgs.adam_lr,
    gamma=TestArgs.gamma,
    minibatch_size=TestArgs.minibatch_size,
    initial_likelihood_var=TestArgs.initial_likelihood_var,
    seed=TestArgs.seed
)

# Fit Model to Data
t0 = time.time()
gp_model.fit(xtrain, ytrain)
t1 = time.time() - t0

print(
    f"Training Time: {t1:.3} seconds"
)

Training Time: 3.38 seconds


In [59]:
# Predict with test set
t0 = time.time()
ypred, ystd = gp_model.predict(xtest, return_std=True)
t1 = time.time() - t0
# Get Stats
mae = mean_absolute_error(ypred, ytest)
mse = mean_squared_error(ypred, ytest)
rmse = np.sqrt(mse)
r2 = r2_score(ypred, ytest)

print(
    f"GP Model:\n"
    f"MAE: {mae:.3f}\nMSE: {mse:.3f}\nRMSE: {rmse:.3f}\nR2: {r2:.3f}" 
    f" \nTime: {t1:.3} seconds"
)

GP Model:
MAE: 15.433
MSE: 549.661
RMSE: 23.445
R2: 0.951 
Time: 0.248 seconds


### MultiOutput

In [63]:
# Good practice
gpflow.reset_default_graph_and_session()

# Initialize Model
mogp_model = MOSVGP(
    num_inducing=TestArgs.num_inducing,
    iterations=TestArgs.iterations,
    small_iterations=TestArgs.small_iterations,
    adam_lr=TestArgs.adam_lr,
    gamma=TestArgs.gamma,
    minibatch_size=TestArgs.minibatch_size,
    initial_likelihood_var=TestArgs.initial_likelihood_var,
    seed=TestArgs.seed
)

# Fit Model to Data
t0 = time.time()
mogp_model.fit(xtrain, ytrain)
t1 = time.time() - t0

print(
    f"Training Time: {t1:.3} seconds"
)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-45642233-434/variance/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/pytho

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-45642233-434/variance/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/pytho

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-45642233-434/lengthscales/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/p

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-45642233-434/lengthscales/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/p

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SVGP-e0dbec03-470/q_mu/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/gpflow/core/compilable.py", line 90, in __init__
    self.build()  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/h

...

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-2c117ee9-437/variance/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/pytho

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-2c117ee9-437/variance/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/pytho

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-2c117ee9-437/lengthscales/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/p

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SquaredExponential-2c117ee9-437/lengthscales/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/emmanuel/.conda/envs/dl_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3316, in run_code
    return outflag  File "<ipython-input-53-d2d7c739ed38>", line 18, in <module>
    gp_model.fit(xtrain, ytrain)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 130, in fit
    self.model = gpflow.models.SVGP(X, Y, kern, lik, feat=feature, minibatch_size=mb_size)  File "/home/emmanuel/projects/2019_ocean/src/models/gpflow_gpu.py", line 123, in <listcomp>
    kern_list = [gpflow.kernels.RBF(X.shape[1], lengthscales=float(X.shape[1])**0.5) for _ in range(p_outputs)]  File "/home/emmanuel/.conda/envs/dl_py36/lib/p

KeyboardInterrupt: 

In [ ]:
# Predict with test set
t0 = time.time()
ypred, ystd = mogp_model.predict(xtest, return_std=True)
t1 = time.time() - t0
# Get Stats
mae = mean_absolute_error(ypred, ytest)
mse = mean_squared_error(ypred, ytest)
rmse = np.sqrt(mse)
r2 = r2_score(ypred, ytest)

print(
    f"GP Model:\n"
    f"MAE: {mae:.3f}\nMSE: {mse:.3f}\nRMSE: {rmse:.3f}\nR2: {r2:.3f}" 
    f" \nTime: {t1:.3} seconds"
)